In [1]:
from DeepLearning.Environments.NoSetupEnv import NoSetupDenseRewardEnv
from DeepLearning.Environments.SetupEnv import SetupRandomWithRoadsEnv
from DeepLearning.Environments.CatanEnv import CatanEnv
from DeepLearning.Environments.SelfPlayEnv import SelfPlayEnv
from sb3_contrib.ppo_mask import MaskablePPO
import os

os.environ["UPDATE_MODELS"] = "False"
os.environ["MODEL_NAME"] = "None"

#env = NoSetupDenseRewardEnv(setupModel=MaskablePPO.load('DeepLearning/Models/SetupRandom_wins_1M.zip'))
env = SelfPlayEnv()

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1, tensorboard_log="./tensorboard_logs/")
model.learn(total_timesteps=5_000_000, tb_log_name='Full_SelfPlay_8vp_5M')
# model.save("DeepLearning/Models/Full_SelfPlay_1M")


486
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard_logs/Full_SelfPlay_8vp_5M_1


/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 324      |
|    ep_rew_mean     | 7.33     |
| time/              |          |
|    fps             | 1110     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 277         |
|    ep_rew_mean          | 6.36        |
| time/                   |             |
|    fps                  | 846         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008911759 |
|    clip_fraction        | 0.118       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.778      |
|    explained_variance   | -1.07       |
|    learning_rate        | 0.

In [2]:
from sb3_contrib.common.maskable.utils import get_action_masks
from DeepLearning.Environments.NoSetupEnv import NoSetupDenseRewardEnv
from DeepLearning.Environments.SetupEnv import SetupRandomWithRoadsEnv
from DeepLearning.Environments.CatanEnv import CatanEnv
import os
from Game.CatanPlayer import PlayerStatsTracker
from sb3_contrib.ppo_mask import MaskablePPO
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentMCTS import AgentMCTS
from tabulate import tabulate
from DeepLearning.Stats import headers
import pandas as pd
import random

# env = NoSetupDenseRewardEnv(setupModel=MaskablePPO.load('DeepLearning/Models/SetupRandom_wins_1M.zip'))
env = CatanEnv()
modelName = "Full_SelfPlay_8vp_5M"

model.save(f"DeepLearning/Models/{modelName}")
# model = MaskablePPO.load(f'DeepLearning/Models/{modelName}.zip')

rewardList = []
winner = [0,0,0,0]

stats = PlayerStatsTracker()
randomStats = PlayerStatsTracker()

players = [ AgentRandom2("P0", 0),
            AgentRandom2("P1", 1),
            AgentRandom2("P2", 2),
            AgentRandom2("P3", 3)]

for episode in range(1000):
    done = False
    state, info = env.reset()#players=players)

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        rewardList.append(reward)

    winner[env.game.gameState.winner] += 1
    
    env.game.gameState.players[0].generatePlayerStats()
    env.game.gameState.players[3].generatePlayerStats()

    stats += env.game.gameState.players[0].stats
    randomStats += env.game.gameState.players[3].stats

# Collect stats
opponentName = "AgentRandom"

stats.getAverages()
randomStats.getAverages()

agentData = stats.getList()
agentData.insert(0, winner[0]/sum(winner))
agentData.insert(0, modelName)
randomData = randomStats.getList()
randomData.insert(0, winner[3]/sum(winner))
randomData.insert(0, opponentName)

table = tabulate([agentData, randomData], headers=headers, tablefmt='simple')
print(table)

# Save to CSV
fileName = f'{modelName}_vs_3{opponentName}.csv'
df = pd.DataFrame([agentData, randomData], columns=headers)
df.to_csv(f'DeepLearning/Data/{fileName}', index=False)


print("\n\nWinnings: ", winner)

# Brick, ore, wool, wheat, wood

SyntaxError: invalid syntax (2572446449.py, line 67)

In [12]:
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentMCTS import AgentMCTS
from Agents.AgentModel import AgentModel
from Game.CatanGame import *
from CatanSimulator import CreateGame
from sb3_contrib.ppo_mask import MaskablePPO
from DeepLearning.Environments.SetupEnv import SetupRandomEnv, SetupRandomWithRoadsEnv, SetupOnlyEnv
from DeepLearning.Environments.NoSetupEnv import NoSetupDenseRewardEnv
import dill as pickle


winner = [0,0,0,0]

players = [ AgentModel("P0", 0, setupModel=MaskablePPO.load("DeepLearning/Models/SetupRandom_wins_1M.zip"), setupEnv=SetupRandomEnv(), model=MaskablePPO.load("DeepLearning/Models/NoSetupDenseRewardEnv-10M.zip"), env=NoSetupDenseRewardEnv()),
            AgentRandom2("P1", 1),
            AgentRandom2("P2", 2),
            AgentRandom2("P3", 3)]
            # AgentModel("P0", 0, setupModel=MaskablePPO.load("DeepLearning/Models/SetupRandom_wins_1M.zip"), setupEnv=SetupRandomEnv(), model=MaskablePPO.load("DeepLearning/Models/NoSetupDenseRewardEnv-10M.zip"), env=NoSetupDenseRewardEnv())
            # AgentModel("P1", 1, setupModel=MaskablePPO.load("DeepLearning/Models/SetupRandom_wins_100k.zip"), setupEnv=SetupRandomEnv(), model=MaskablePPO.load("DeepLearning/Models/NoSetupDenseRewardEnv-10M.zip"), env=NoSetupDenseRewardEnv()),
            # AgentModel("P2", 2, setupModel=MaskablePPO.load("DeepLearning/Models/SetupRandomRoads_wins_1M.zip"), setupEnv=SetupRandomWithRoadsEnv(), model=MaskablePPO.load("DeepLearning/Models/NoSetupDenseRewardEnv-10M.zip"), env=NoSetupDenseRewardEnv()),
            # AgentModel("P3", 3, setupModel=MaskablePPO.load("DeepLearning/Models/SetupOnly_DotTotal_100k.zip"), setupEnv=SetupOnlyEnv(), model=MaskablePPO.load("DeepLearning/Models/NoSetupDenseRewardEnv-10M.zip"), env=NoSetupDenseRewardEnv())]

for episode in range(1000):
    game = CreateGame(players)
    game = pickle.loads(pickle.dumps(game, -1))
    while True:
        currPlayer = game.gameState.players[game.gameState.currPlayer]

        agentAction = currPlayer.DoMove(game)
        agentAction.ApplyAction(game.gameState)

        if game.gameState.currState == "OVER":
            break
    winner[game.gameState.winner] += 1


print("\n\nWinnings: ", winner)

# Brick, ore, wool, wheat, wood



Winnings:  [960, 12, 13, 15]
